# Pyspark With Python-Introduction To Pyspark Mlib

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Mlib').getOrCreate()

In [3]:
spark

## Loading Data Set

In [6]:
#Reading the data
df_spark = spark.read.csv('file_example_3.csv',header=True,inferSchema=True)
df_spark.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [7]:
df_spark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [8]:
df_spark.columns

['Name', 'age', 'Experience', 'Salary']

# Data Pre-Processing

In [12]:
from pyspark.ml.feature import VectorAssembler
feature_assemble = VectorAssembler(inputCols=['age','Experience'],
                                  outputCol='Independent_feature')

In [13]:
output = feature_assemble.transform(df_spark)

In [14]:
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|Independent_feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [15]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent_feature']

In [17]:
final_data = output.select('Independent_feature','Salary')
final_data.show()

+-------------------+------+
|Independent_feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



# Model Building

In [18]:
from pyspark.ml.regression import LinearRegression
train_data,test_data = final_data.randomSplit([0.70,0.30])
regression = LinearRegression(featuresCol='Independent_feature',labelCol='Salary')
regression = regression.fit(train_data)

In [19]:
### Coefficients
regression.coefficients

DenseVector([-258.8832, 1670.0508])

In [20]:
### Intercepts
regression.intercept

20543.147208121274

In [21]:
## Prediction
predicted_result = regression.evaluate(test_data)

In [22]:
predicted_result.predictions.show()

+-------------------+------+------------------+
|Independent_feature|Salary|        prediction|
+-------------------+------+------------------+
|         [21.0,1.0]| 15000|16776.649746192812|
|         [24.0,3.0]| 20000|19340.101522842593|
+-------------------+------+------------------+



In [23]:
predicted_result.meanAbsoluteError

1218.2741116751095

In [24]:
predicted_result.r2

0.7126439743358681